# Compare CRS Data to IATI Transactions

# Setup

Donor mapping and CRS aggregated output source: https://github\.com/akmiller01/ODSC\-CRS\-ODA\-comp/tree/main, 13/1/25

Read in files \- using data generated in Disbursements & Expenditures vs Flow

In [1]:
import pandas as pd
import numpy as np

In [2]:
crs_data = pd.read_csv("crs_aggregated_2014_2023.csv")
mapping = pd.read_csv("iati_donor_mapping_20241101.csv")
disb_exp_flow = pd.read_excel("dispexpvsflow_data.xlsx")

# Map IATI organisations to CRS

Mapping source: https://github\.com/akmiller01/ODSC\-CRS\-ODA\-comp/blob/main/iati\_donor\_mapping\_20241101\.csv

In [3]:
iati_mapped = mapping.merge(disb_exp_flow, left_on="reporting_org_ref",right_on="reportingorg_ref",how='left')
iati_mapped = iati_mapped.rename(columns={'comb_flowtype_code': 'category_code', 'comb_flowtype_codename': 'category_name'})
iati_mapped

,reporting_org_ref,publisher_name,donor_code,donor_name,comments,iatiidentifier,reportingorg_ref,transactiontype_code,transactiontype_codename,year,sum_value_usd,flowtype_code,flowtype_codename,defaultflowtype_code,defaultflowtype_codename,category_code,category_name
0,XM-DAC-2-10,Belgian Development Cooperation,2,Belgium,NaN,XM-DAC-2-10-1012,XM-DAC-2-10,3.0,Disbursement,2016.0,4.327568e+04,NaN,NaN,10.0,ODA,10.0,ODA
1,XM-DAC-2-10,Belgian Development Cooperation,2,Belgium,NaN,XM-DAC-2-10-1012,XM-DAC-2-10,3.0,Disbursement,2017.0,3.811171e+05,NaN,NaN,10.0,ODA,10.0,ODA
2,XM-DAC-2-10,Belgian Development Cooperation,2,Belgium,NaN,XM-DAC-2-10-1012,XM-DAC-2-10,3.0,Disbursement,2018.0,1.320229e+06,NaN,NaN,10.0,ODA,10.0,ODA
3,XM-DAC-2-10,Belgian Development Cooperation,2,Belgium,NaN,XM-DAC-2-10-1012,XM-DAC-2-10,3.0,Disbursement,2019.0,8.491214e+05,NaN,NaN,10.0,ODA,10.0,ODA
4,XM-DAC-2-10,Belgian Development Cooperation,2,Belgium,NaN,XM-DAC-2-10-1012,XM-DAC-2-10,3.0,Disbursement,2020.0,6.749012e+05,NaN,NaN,10.0,ODA,10.0,ODA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805441,US-EIN-941655673,The William and Flora Hewlett Foundation,1624,William and Flora Hewlett Foundation,NaN,US-EIN-941655673-2024-04803-GRA,US-EIN-941655673,3.0,Disbursement,2024.0,1.100000e+05,NaN,NaN,30.0,Private Development Finance,30.0,Private Development Finance
805442,US-EIN-941655673,The William and Flora Hewlett Foundation,1624,William and Flora Hewlett Foundation,NaN,US-EIN-941655673-2024-04833-GRA,US-EIN-941655673,3.0,Disbursement,2024.0,1.500000e+05,NaN,NaN,30.0,Private Development Finance,30.0,Private Development Finance
805443,US-EIN-941655673,The William and Flora Hewlett Foundation,1624,William and Flora Hewlett Foundation,NaN,US-EIN-941655673-2024-04860-GRA,US-EIN-941655673,3.0,Disbursement,2024.0,4.000000e+04,NaN,NaN,30.0,Private Development Finance,30.0,Private Development Finance
805444,US-EIN-941655673,The William and Flora Hewlett Foundation,1624,William and Flora Hewlett Foundation,NaN,US-EIN-941655673-2024-04871-GRA,US-EIN-941655673,3.0,Disbursement,2024.0,2.500000e+05,NaN,NaN,30.0,Private Development Finance,30.0,Private Development Finance


# Aggregate

Group IATI transaction values by year, donor, and flow type\.

In [4]:
iati_mapped_agg = iati_mapped.groupby(['year','donor_code', 'donor_name','category_code','category_name'])['sum_value_usd'].sum().reset_index()
iati_mapped_agg['iati_usd_mil'] = iati_mapped_agg['sum_value_usd']/1000000
iati_mapped_agg = iati_mapped_agg.drop('sum_value_usd', axis=1)
iati_mapped_agg

,year,donor_code,donor_name,category_code,category_name,iati_usd_mil
0,2010.0,2,Belgium,10.0,ODA,229.873999
1,2010.0,3,Denmark,10.0,ODA,1880.802198
2,2010.0,5,Germany,10.0,ODA,456.390627
3,2010.0,6,Italy,10.0,ODA,16.164210
4,2010.0,7,Netherlands,10.0,ODA,3824.749728
...,...,...,...,...,...,...
714,2024.0,1050,WHO-Strategic Preparedness and Response Plan,10.0,ODA,1549.641596
715,2024.0,1311,Global Alliance for Vaccines and Immunization,10.0,ODA,2154.921850
716,2024.0,1312,Global Fund,10.0,ODA,4378.750510
717,2024.0,1601,Bill & Melinda Gates Foundation,30.0,Private Development Finance,3323.404193


In [5]:
iati_mapped_agg.to_csv("iati_donor_disb_exp_by_flowtype_2014_2023.csv", index=False)

Group IATI transaction values by year and flow type\.

In [6]:
iati_year_aidtype = iati_mapped.groupby(['year','category_code','category_name'])['sum_value_usd'].sum().reset_index()
iati_year_aidtype['iati_usd_mil'] = iati_year_aidtype['sum_value_usd']/1000000
iati_year_aidtype = iati_year_aidtype.drop('sum_value_usd', axis=1)
iati_year_aidtype

,year,category_code,category_name,iati_usd_mil
0,2010.0,10.0,ODA,30346.133150
1,2010.0,20.0,OOF,17507.870978
2,2010.0,21.0,Non-export credit OOF,1384.733220
3,2010.0,30.0,Private Development Finance,107.566545
4,2010.0,35.0,Private Market,0.000000
...,...,...,...,...
84,2024.0,10.0,ODA,180948.599135
85,2024.0,20.0,OOF,20982.110898
86,2024.0,21.0,Non-export credit OOF,9530.504503
87,2024.0,30.0,Private Development Finance,3540.760697


Group CRS data by year and flow type\.

In [7]:
crs_data_agg = crs_data.groupby(['year','category_code','category_name'])['value'].sum().reset_index()
crs_data_agg

,year,category_code,category_name,value
0,2014,10,ODA,166173.154968
1,2014,21,Non-export credit OOF,56034.880130
2,2014,30,Private Development Finance,2849.778452
3,2015,10,ODA,167463.479341
4,2015,21,Non-export credit OOF,71561.232181
5,2015,30,Private Development Finance,3242.273631
6,2016,10,ODA,181520.878808
7,2016,21,Non-export credit OOF,64844.249058
8,2016,30,Private Development Finance,4062.015936
9,2017,10,ODA,191479.739666


# Join and compare

Compare IATI transactions to CRS data, looking at year, donor, and flow type\.

In [8]:
compare = crs_data.merge(iati_mapped_agg, on=['year','donor_code', 'donor_name','category_code','category_name'],how='outer')
compare

,year,donor_code,donor_name,category_code,category_name,value,iati_usd_mil
0,2010.0,2.0,Belgium,10.0,ODA,NaN,229.873999
1,2010.0,3.0,Denmark,10.0,ODA,NaN,1880.802198
2,2010.0,5.0,Germany,10.0,ODA,NaN,456.390627
3,2010.0,6.0,Italy,10.0,ODA,NaN,16.164210
4,2010.0,7.0,Netherlands,10.0,ODA,NaN,3824.749728
...,...,...,...,...,...,...,...
2011,2024.0,1050.0,WHO-Strategic Preparedness and Response Plan,10.0,ODA,NaN,1549.641596
2012,2024.0,1311.0,Global Alliance for Vaccines and Immunization,10.0,ODA,NaN,2154.921850
2013,2024.0,1312.0,Global Fund,10.0,ODA,NaN,4378.750510
2014,2024.0,1601.0,Bill & Melinda Gates Foundation,30.0,Private Development Finance,NaN,3323.404193


In [9]:
compare.to_csv("crs_iati_donor_disb_exp_by_flowtype_2014_2023_comparison.csv", index=False)

In [11]:
compare['percent_difference'] = (compare['iati_usd_mil'] / compare['value']) * 100

In [18]:
compare

,year,donor_code,donor_name,category_code,category_name,value,iati_usd_mil,percent_difference
0,2010.0,2.0,Belgium,10.0,ODA,NaN,229.873999,NaN
1,2010.0,3.0,Denmark,10.0,ODA,NaN,1880.802198,NaN
2,2010.0,5.0,Germany,10.0,ODA,NaN,456.390627,NaN
3,2010.0,6.0,Italy,10.0,ODA,NaN,16.164210,NaN
4,2010.0,7.0,Netherlands,10.0,ODA,NaN,3824.749728,NaN
...,...,...,...,...,...,...,...,...
2011,2024.0,1050.0,WHO-Strategic Preparedness and Response Plan,10.0,ODA,NaN,1549.641596,NaN
2012,2024.0,1311.0,Global Alliance for Vaccines and Immunization,10.0,ODA,NaN,2154.921850,NaN
2013,2024.0,1312.0,Global Fund,10.0,ODA,NaN,4378.750510,NaN
2014,2024.0,1601.0,Bill & Melinda Gates Foundation,30.0,Private Development Finance,NaN,3323.404193,NaN


In [13]:
compare.to_csv("crs_iati_donor_disb_exp_by_flowtype_2014_2023_comparison.csv", index=False)

Compare donor grouping aggregation across of flow-types. 

In [17]:
import numpy as np

donor_group_compare = (
    compare
    .groupby(["year", "donor_code", "donor_name"], as_index=False)
    .agg(
        value=("value", "sum"),
        iati_usd_mil=("iati_usd_mil", "sum")
    )
)

donor_group_compare["percent_difference"] = np.where(
    out["value"] > 0,
    (out["iati_usd_mil"] / out["value"]) * 100,
    np.nan
)

donor_group_compare


,year,donor_code,donor_name,value,iati_usd_mil,percent_difference
0,2010.0,2.0,Belgium,0.0,229.873999,NaN
1,2010.0,3.0,Denmark,0.0,1880.802198,NaN
2,2010.0,5.0,Germany,0.0,456.390627,NaN
3,2010.0,6.0,Italy,0.0,16.164210,NaN
4,2010.0,7.0,Netherlands,0.0,3840.203413,NaN
...,...,...,...,...,...,...
1508,2024.0,1050.0,WHO-Strategic Preparedness and Response Plan,0.0,1549.641596,NaN
1509,2024.0,1311.0,Global Alliance for Vaccines and Immunization,0.0,2154.921850,NaN
1510,2024.0,1312.0,Global Fund,0.0,4378.750510,NaN
1511,2024.0,1601.0,Bill & Melinda Gates Foundation,0.0,3323.404193,NaN


In [19]:
donor_group_compare.to_csv("crs_iati_donor_disb_exp_comparison_2014_2023.csv", index=False)

Compare IATI transactions to CRS data, looking at year and flow type\.

In [10]:
compare_yearaid = crs_data_agg.merge(iati_year_aidtype, on=['year','category_code','category_name'],how='outer')
compare_yearaid['perc'] = (compare_yearaid['iati_usd_mil']/compare_yearaid['value'])*100
compare_yearaid

,year,category_code,category_name,value,iati_usd_mil,perc
0,2010.0,10.0,ODA,NaN,30346.133150,NaN
1,2010.0,20.0,OOF,NaN,17507.870978,NaN
2,2010.0,21.0,Non-export credit OOF,NaN,1384.733220,NaN
3,2010.0,30.0,Private Development Finance,NaN,107.566545,NaN
4,2010.0,35.0,Private Market,NaN,0.000000,NaN
...,...,...,...,...,...,...
85,2024.0,10.0,ODA,NaN,180948.599135,NaN
86,2024.0,20.0,OOF,NaN,20982.110898,NaN
87,2024.0,21.0,Non-export credit OOF,NaN,9530.504503,NaN
88,2024.0,30.0,Private Development Finance,NaN,3540.760697,NaN
